In [1]:
!pip -q install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
!pip install -q einops accelerate langchain bitsandbytes
!pip -q install  xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 5.7 MB/s eta 0:00:00


In [2]:
!pip install -q auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


## Mistral 7B Instruct



In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(




*** Generate:
<s><s>[INST] Tell me about AI [/INST] AI (Artificial Intelligence) is a branch of computer science that focuses on the development of intelligent machines that work and react like humans. AI technologies include machine learning, natural language processing, robotics, and computer vision. AI is used in a variety of applications, including virtual assistants, self-driving cars, speech recognition, and image recognition. AI has the potential to revolutionize many industries, including healthcare, finance, and education.</s>
*** Pipeline:
<s>[INST] Tell me about AI [/INST] Artificial intelligence (AI) refers to the development of computer systems and machines that are able to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be classified into different categories based on its level of autonomy, including narrow or weak AI, which is designed for a specific task, and gene

In [6]:
prompt = "Write a detailed analogy between mathematics and a lighthouse"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] Write a detailed analogy between mathematics and a lighthouse [/INST] Mathematics can be compared to a lighthouse in many ways. Just like how a lighthouse guides ships safely to the shore by emitting a beam of light, mathematics provides a set of rules and guidelines that help people navigate through complex problems and make informed decisions.

The tower of the lighthouse represents the foundations of mathematics, which provide a strong base for understanding and solving problems. The light that is emitted from the lighthouse represents the various mathematical concepts and formulas that are used to guide ships to safety. These concepts include trigonometry, calculus, geometry, and statistics, among others.

The beacon of the lighthouse can be thought of as the solution to a problem. It is the result of applying the mathematical rules and guidelines in a specific way. Similarly, the solution to a mathematical problem is the result of using the appropriate mathematical conce

In [7]:
prompt = "Write a detailed analogy between mathematics and a music."
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] Write a detailed analogy between mathematics and a music. [/INST] Mathematics and music are two distinct forms of art that can be compared in several ways. Just as music is a language that uses melody, rhythm, and harmony to convey emotions and tell stories, mathematics is a language that uses numbers, symbols, and formulas to describe and explain the world around us. 

One way to compare the two is through their structure. Both mathematics and music have a natural progression that follows a set of rules or principles. In music, this might mean following a particular chord progression or scale, while in mathematics it means following the rules of algebra or geometry. Both art forms require practice and patience to master, and both offer a sense of satisfaction and fulfillment when understood and executed correctly.

Another similarity between mathematics and music is their ability to evoke emotion. Just as different chords or melodies can elicit specific feelings or moods, ma

In [8]:
prompt = "Write a short email to Sam Altman giving reasons to open source GPT-4"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] Write a short email to Sam Altman giving reasons to open source GPT-4 [/INST] Subject: Reasons to Open Source GPT-4

Dear Sam,

I hope this message finds you well. I am writing to express my strong belief that open sourcing GPT-4 would be beneficial for the community and the field of AI research as a whole. Here are some specific reasons why I think it is important:

1. Scientific Advancement: By making GPT-4 open source, researchers can build upon each other's work, share new ideas, and collaborate more effectively. This can lead to rapid progress in the field and the development of even more advanced models.
2. Accessibility: Currently, access to powerful AI models like GPT-4 is limited to those with significant resources and expertise. Open sourcing the model would democratize access to these tools and allow researchers from all backgrounds to contribute to the field.
3. Transparency: Open source allows for greater transparency in the development and use of AI models. This

In [9]:
prompt = "What is the capital of England?"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] What is the capital of England? [/INST] The capital city of England is London.


In [11]:
prompt = """Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:"""
#prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt)[0]['generated_text'])

Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? 

 Bob: Well, have you tried breaking up your study sessions into smaller chunks and taking breaks in between? It can help you maintain focus and increase productivity.

Alice: That is a great idea! But sometimes I get overwhelmed with the amount of material I need to cover. How do I prioritize what to study first?

Bob: One way to prioritize would be to create a study schedule or plan that outlines what topics you will cover each day or week. This will help you stay on track and avoid feeling overwhelmed. You could also try focusing on one topic at a time and only moving on to the next once you feel like you have a good grasp on it.

Alice: Thanks for the advice, Bob. I think creating a study plan will definitely help me stay focused and productive. Do you have any tips for staying motivated during exam season?

Bob: One tip is to set achievable goals for yourself, both short-term and long-term. 

## GSM8K

In [12]:
prompt = "Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have? [/INST] Let's reason through this problem:

1. The cafeteria started with 23 apples.
2. They used 20 of these apples for lunch.
3. This means that they are left with 3 apples (23 - 20 = 3).
4. Then, they bought an additional 6 apples.
5. Adding these new apples to the remaining ones gives us a total number of apples: 3 + 6 = 9.

So, the cafeteria has 9 apples after using some for lunch and buying more.


In [13]:
prompt = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? [/INST] To calculate how much Weng earned for her babysitting work yesterday, we need to multiply her hourly rate by the number of hours she worked.

First, let's convert the 50 minutes of babysitting into hours:

50 minutes = 50/60 hours = 0.8333 hours (rounded)

Now, we can multiply Weng's hourly rate by the number of hours she worked to find out how much she earned:

Earnings = Hourly rate x Number of hours
= $12 per hour x 0.8333 hours
≈ $10


In [14]:
prompt = "A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?"
prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt_template)[0]['generated_text'])

<s>[INST] A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years? [/INST] Let's work through this problem step by step:
1. We know that the monster consumes 847 people every three hundred years.
2. Since there are 100 years between each feeding, we can assume that the monster consumes 847/3 = 282 people per feeding.
3. However, we need to find out how many people were on the ship the monster ate in the first hundred years.
4. Since ships have been built larger over time, each new ship has twice as many people as the last ship.
5. Let's denote the number of people on the first ship as x. Then, the number of people on the second ship is 2x, the number of people on the third ship is 2 * 2x = 4x, and so on.
6. The